# Bag-of-Words-Meet-Bags-of-Popcorn

IMOPRTING DATA FOR TRAINING AND TESTING

In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0,delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0,delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews,and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



 DATA PREPROCESSING

In [2]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

Download the punkt tokenizer for sentence splitting and loading the data

In [3]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()   

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [4]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Defining a function to split a review into parsed sentences

In [5]:
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [6]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

This is just to be done once, after this built model can be used

In [64]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "W2V_MODEL"
model.save(model_name)

2018-10-10 08:11:08,857 : INFO : collecting all words and their counts
2018-10-10 08:11:08,859 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-10 08:11:08,928 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-10-10 08:11:09,002 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2018-10-10 08:11:09,080 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-10-10 08:11:09,158 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-10-10 08:11:09,236 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-10-10 08:11:09,315 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-10-10 08:11:09,397 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-10-10 08:11:09,478 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-10-10 08:11:09,558 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-10-10 08:11:09,634 : INFO : PROGRESS: at sentence #100000, processed 2226967 words, keeping 50207 word types
2018-10-10 08:11:09,713 : INFO : PROGRESS: at sentence #110000, processed 2446581 words, keeping 

2018-10-10 08:11:14,911 : INFO : PROGRESS: at sentence #750000, processed 16771406 words, keeping 120295 word types
2018-10-10 08:11:14,991 : INFO : PROGRESS: at sentence #760000, processed 16990810 words, keeping 120930 word types
2018-10-10 08:11:15,080 : INFO : PROGRESS: at sentence #770000, processed 17217947 words, keeping 121703 word types
2018-10-10 08:11:15,164 : INFO : PROGRESS: at sentence #780000, processed 17448093 words, keeping 122402 word types
2018-10-10 08:11:15,246 : INFO : PROGRESS: at sentence #790000, processed 17675169 words, keeping 123066 word types
2018-10-10 08:11:15,294 : INFO : collected 123504 word types from a corpus of 17798270 raw words and 795538 sentences
2018-10-10 08:11:15,296 : INFO : Loading a fresh vocabulary
2018-10-10 08:11:15,440 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-10-10 08:11:15,441 : INFO : effective_min_count=40 leaves 17239125 word corpus (96% of original 17798270, drops 559

2018-10-10 08:12:09,082 : INFO : EPOCH 3 - PROGRESS: at 58.57% examples, 614152 words/s, in_qsize 7, out_qsize 0
2018-10-10 08:12:10,092 : INFO : EPOCH 3 - PROGRESS: at 63.39% examples, 613740 words/s, in_qsize 7, out_qsize 0
2018-10-10 08:12:11,109 : INFO : EPOCH 3 - PROGRESS: at 68.26% examples, 613686 words/s, in_qsize 7, out_qsize 0
2018-10-10 08:12:12,121 : INFO : EPOCH 3 - PROGRESS: at 73.00% examples, 612831 words/s, in_qsize 7, out_qsize 0
2018-10-10 08:12:13,127 : INFO : EPOCH 3 - PROGRESS: at 77.83% examples, 612725 words/s, in_qsize 7, out_qsize 0
2018-10-10 08:12:14,139 : INFO : EPOCH 3 - PROGRESS: at 82.77% examples, 613241 words/s, in_qsize 7, out_qsize 0
2018-10-10 08:12:15,166 : INFO : EPOCH 3 - PROGRESS: at 87.74% examples, 613640 words/s, in_qsize 6, out_qsize 1
2018-10-10 08:12:16,168 : INFO : EPOCH 3 - PROGRESS: at 92.56% examples, 613669 words/s, in_qsize 6, out_qsize 1
2018-10-10 08:12:17,169 : INFO : EPOCH 3 - PROGRESS: at 97.46% examples, 614069 words/s, in_qsiz

In [7]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [8]:
from gensim.models import Word2Vec
model = Word2Vec.load("W2V_MODEL")

Function to average all of the word vectors in a given paragraph

In [66]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.true_divide(featureVec,nwords)
    return featureVec

This function calculates the average feature vector for one and returns a 2D array

In [65]:
def getAvgFeatureVecs(reviews,model,num_features):
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000. == 0.:
            print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

Calculate average feature vectors for training and testing sets,using the functions defined above

In [68]:
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review,remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review,remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews,model,num_features )

Review 0 of 25000


/home/guru_dsce11/.local/lib/python3.5/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

# Once the vectors are ready it has to be trained .Here MLPs are used

### MLPs

In [85]:
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import RandomNormal
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.preprocessing import sequence

+ The Sequential model is a linear stack of layers.
+ It can be created by passing a list of layer instances to the constructor:
+ The layers can be created via .add() method
+ Dense implements the operation: output = activation(dot(input, kernel) + bias) where
+ activation is the element-wise activation function passed as the activation argument,there are many activation functions ar available ex: tanh, relu, softmax 
+ kernel is a weights matrix created by the layer, and 
+ bias is a bias vector created by the layer (only applicable if use_bias is True).

In [86]:
model = Sequential()

model.add(Dense(512, activation='relu',input_shape=(300,),kernel_initializer=RandomNormal(mean=0.0, stddev=0.062, seed=None)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu',input_shape=(300,),kernel_initializer=RandomNormal(mean=0.0, stddev=0.062, seed=None)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu',input_shape=(300,),kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)) )
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.save("MLP_model")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 512)               154112    
_________________________________________________________________
batch_normalization_11 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 128)               65664     
_________________________________________________________________
batch_normalization_12 (Batc (None, 128)               512       
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 64)                8256      
__________

In [87]:
model.fit( trainDataVecs, train["sentiment"],epochs=20,batch_size=64)

Epoch 1/20
25000/25000 [==============================] - 7s 261us/step - loss: 0.5137 - acc: 0.7631
Epoch 2/20
25000/25000 [==============================] - 3s 131us/step - loss: 0.3764 - acc: 0.8422
Epoch 3/20
25000/25000 [==============================] - 3s 132us/step - loss: 0.3588 - acc: 0.8513
Epoch 4/20
25000/25000 [==============================] - 3s 131us/step - loss: 0.3447 - acc: 0.8572
Epoch 5/20
25000/25000 [==============================] - 3s 128us/step - loss: 0.3467 - acc: 0.8562
Epoch 6/20
25000/25000 [==============================] - 3s 129us/step - loss: 0.3352 - acc: 0.8617
Epoch 7/20
25000/25000 [==============================] - 3s 128us/step - loss: 0.3388 - acc: 0.8582
Epoch 8/20
25000/25000 [==============================] - 3s 129us/step - loss: 0.3324 - acc: 0.8606
Epoch 9/20
25000/25000 [==============================] - 3s 130us/step - loss: 0.3303 - acc: 0.8620
Epoch 10/20
25000/25000 [==============================] - 3s 131us/step - loss: 0.3274 - a

In [185]:
result=model.predict(testDataVecs)

In [193]:
for i in range(0,len(result)):
    if(result[i]>0.5):
        result[i]=1
    else:
        result[i]=0
    

In [188]:
result=result.ravel()

In [189]:
result=result.astype(int)

In [192]:
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "output.csv", index=False, quoting=3 )